<a href="https://colab.research.google.com/github/gtbnhyujmj/BingX_Auto_Order/blob/main/BingX_Bot_Me_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install requests

# 公鑰與密鑰

In [ ]:
APIKEY = ""
SECRETKEY = ""

# 套件

In [ ]:
import time                # 引入時間模組，用來取得現在時間
import requests              # 引入requests模組，讓我們可以發送網路請求
import hmac                # 引入hmac模組，幫忙做簽名加密
from hashlib import sha256        # 引入sha256加密演算法

# 定義主程式

In [ ]:
def demo():
    # 這個函式示範怎麼叫API查帳戶餘額
    path = 'https://open-api.bingx.com/openApi/spot/v1/account/balance'     # API的路徑 V1
    method = "GET"                                # 用GET方法請求資料

    # A = params["recvWindow"]
    # B = params["timestamp"]

    # params_str = "recvWindow=" + A + "&" + "timestamp=" + B   # 把字典格式的參數變成字串

    return method, path, # params_str  # 呼叫發送請求的函式，並回傳結果

method, path = demo()
demo()

('GET', 'https://open-api.bingx.com/openApi/spot/v1/account/balance')

# Params

In [ ]:
params = {
        "recvWindow": "60000",                    # 允許的請求時間誤差，60秒（毫秒）
        "timestamp": str(int(time.time() * 1000))          # 現在時間戳記（毫秒），用來驗證請求有效
    }

# 黏貼Params

In [ ]:
def format_params(params):
    # >>> 這個函式把參數排好序，然後組成a=1&b=2這樣的字串

    keys = sorted(params.keys())  # 把參數的鍵排序

    # 建立一個空清單準備放字串
    param_list = []
    for key in keys:
        # 這裡用簡單字串串接，把key跟value串成 "key=value" 的形式
        param_list.append(key + "=" + params[key])

    # 用 & 把所有參數連起來
    return "&".join(param_list)

final_paramas = format_params(params)
format_params(params)  ## 這個就是網頁上的 params_str = 把參數排好序，然後組成a=1&b=2這樣的字串

'recvWindow=60000&timestamp=1747285392505'

# 密鑰

In [ ]:
SECRETKEY = ""

# 用密鑰簽證

In [ ]:
def get_signature(secret, message): # message = 上面處理好的字串
    # 這裡用HMAC-SHA256演算法，根據密鑰跟參數字串做簽名

    sign = hmac.new(secret.encode(), message.encode(), sha256).hexdigest()
    # print("簽名:", sign)      # 印出簽名結果方便檢查
    return sign           # 回傳簽名字串

sign = get_signature(SECRETKEY, final_paramas)
get_signature(SECRETKEY, final_paramas)
# print("簽名:", get_signature)

'0923a2bb14fa4e1ff0d6f6736ad5c65505fad37bd94ffc60e6eb9c053598f6ac'

# 組合成完整網址

In [ ]:
# 把完整網址組合起來（字串串接方式）
url = path + "?" + final_paramas + "&signature=" + sign

# url

'https://open-api.bingx.com/openApi/spot/v1/account/balance?recvWindow=60000&timestamp=1747285392505&signature=0923a2bb14fa4e1ff0d6f6736ad5c65505fad37bd94ffc60e6eb9c053598f6ac'

# 用公鑰設定標頭

In [ ]:
APIKEY = ""

In [ ]:
# 設定HTTP請求的標頭，把APIKEY放進去
headers = {
        'X-BX-APIKEY': APIKEY
    }

# 請求伺服器與取得回音

In [ ]:
# 用requests送出HTTP請求
# method = GET, 函數上面有

response = requests.request(method, url, headers=headers)
response.status_code

200

# 列印JSON

In [ ]:
response = requests.request(method, url, headers=headers)
response.text

'{"code":0,"msg":"","debugMsg":"","data":{"balances":[{"asset":"USDT","disPlayName":"USDT","free":"3.4371838e-10","locked":"0"},{"asset":"USDC","disPlayName":"USDC","free":"0","locked":"0"},{"asset":"LINA","disPlayName":"LINA","free":"0","locked":"0"},{"asset":"ETH","disPlayName":"ETH","free":"0","locked":"0"},{"asset":"POL","disPlayName":"POL","free":"0","locked":"0"}]}}'

In [ ]:
# 自動排版

import json  # 加在檔案開頭

# 在 print(result) 改成這樣：
parsed = json.loads(response.text)
print(json.dumps(parsed, indent=2))

{
  "code": 0,
  "msg": "",
  "debugMsg": "",
  "data": {
    "balances": [
      {
        "asset": "USDT",
        "disPlayName": "USDT",
        "free": "3.4371838e-10",
        "locked": "0"
      },
      {
        "asset": "USDC",
        "disPlayName": "USDC",
        "free": "0",
        "locked": "0"
      },
      {
        "asset": "LINA",
        "disPlayName": "LINA",
        "free": "0",
        "locked": "0"
      },
      {
        "asset": "ETH",
        "disPlayName": "ETH",
        "free": "0",
        "locked": "0"
      },
      {
        "asset": "POL",
        "disPlayName": "POL",
        "free": "0",
        "locked": "0"
      }
    ]
  }
}


# 結論與疑惑

In [ ]:
# USDC是對的
# USDT的數量怎麼有點少?

In [ ]:
# V1 是控制資金帳戶的資料
# 從資金帳戶移動USDT會改變 而且 數字是對的

# "disPlayName": "USDT",
# "free": "0.15684562034371838",

# 把U/C轉回資金帳戶有數字 數字是對的

# V1 控制的是資金帳戶
# V2 控制永續U合約 USDT OK / USDC NOT OK

In [ ]:
# STU